# Skin lesion classification

**Deadline**: Upload this notebook (rename it as 'TP1-ML-YOUR-SURNAME.ipynb') to Ecampus/Moodle before the deadline.


**Context**
A skin lesion is defined as a superficial growth or patch of the skin that is visually different and/or has a different texture than its surrounding area. Skin lesions, such as moles or birthmarks, can degenerate and become melanoma, one of the deadliest skin cancer. Its incidence has been increasing during the last decades, especially in the areas mostly populated by white people.

The most effective treatment is an early detection followed by surgical excision. This is why several approaches for melanoma detection have been proposed in the last years (non-invasive computer-aided diagnosis (CAD) ).

**Data**
You will have at your disposal the ISIC 2017 dataset (https://challenge.isic-archive.com/data/#2017) already pre-processed, resized and quality checked. It is divided into Training (N=2000), Validation (N=150) and Test (N=600) sets.

**Goal**
The goal of this practical session is to classify images of skin lesions as either benign (nevus or seborrheic_keratosis) or melanoma (binary classification) using machine and deep learning algorithms.

In the first part of the TP, you will manually compute some features relevant to the skin lesion classification (feature engineering) and then classify images using "classical" ML algorithms such as, logistic regression, SVM and Random Forests.

In the second part, you will test the features learnt with Deep Learning algorithms. You will first train from scratch well-known CNN architectures (VGG, ResNet, DenseNet, etc..) and then leverage the representations learnt by these networks on a pre-training from Imagenet (fine-tuning, full-restimation).

Please complete the code where you see **"XXXXXXXXX"** and answer the **Questions**


# Feature Engineering

Many features have been proposed for Skin lesion classification. Among the most used ones, there is the so-called ABCD rule whose features describie four important characteristics of the skin lesion: Asymmetry, Border irregularity, Colour(and Texture) and Dimension (and Geometry).

To compute these features, you will have at your disposal the *manual segmentation* of the skin lesions and you could follow, for instance, *Ganster et al. 'Automated melanoma recognition', IEEE TMI, 2001* and *Zortea et al. 'Performance of a dermoscopy-based computer vision system for the diagnosis of pigmented skin lesions compared with visual evaluation by experienced dermatologists', Artificial Intelligence in Medicine, 2014*.

Other works can be found in the literature.

For the ML part, you can use the CPU server (no need for GPU here)

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.io import imsave
from skimage.transform import resize
from skimage import color
from skimage import measure
from skimage import transform
from skimage.color import rgb2gray
from scipy import ndimage
from scipy import stats
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import torch
import glob
import cv2
from PIL import Image


%matplotlib inline

try:
  import google.colab
  IN_COLAB = True
  !pip install gdown==4.6.0 # with the following versions, there is an error
except:
  IN_COLAB = False

You can either download the data from my Google Drive or work locally.

In [ ]:
if IN_COLAB:
  print("you are using google colab")
  import gdown
  !mkdir ./data
  gdown.download(id="1iH5hkRN0wCgGklUN5df9u2Ue3UXAR4xZ", output='./data/TrainCropped.zip', quiet=False)
  !unzip -qu "./data/TrainCropped.zip" -d "./data"
  gdown.download(id="1lyRZuV9UST55AEqwSy4mqMmh5yHGI1FM", output='./data/TestCropped.zip', quiet=False)
  !unzip -qu "./data/TestCropped.zip" -d "./data"
  gdown.download(id="1RLJOmqAnHCgiJ7qShQurpxNaRhjjPpJb", output='./data/ValCropped.zip', quiet=False)
  !unzip -qu "./data/ValCropped.zip" -d "./data"
  !rm -rf ./data/TrainCropped.zip
  !rm -rf ./data/TestCropped.zip
  !rm -rf ./data/ValCropped.zip
  path='./data/'
else:
  print('You are NOT using colab')
  # we assume that folders of data are in the same folder as this jupyter notebook
  path='' # if you change this path , you should also change idTRain, idVal and idTest


If there is an error (might happen with gdown) please upload the three files manually.
Follow the following instructions:
- go to the folder symbol on the left of your screen
- click on the three vertical dots on the 'data' folder
- upload (importer in french) the three folders
That's it !

In [ ]:
# if IN_COLAB:
#   !unzip -qu "./data/TrainCropped.zip" -d "./data"
#   !unzip -qu "./data/TestCropped.zip" -d "./data"
#   !unzip -qu "./data/ValCropped.zip" -d "./data"
#   !rm -rf ./data/TrainCropped.zip
#   !rm -rf ./data/TestCropped.zip
#   !rm -rf ./data/ValCropped.zip
#   path='./data/'

Let's load the data.

In [ ]:
pathTrain=glob.glob(path + "TrainCropped/*.jpg")
print(pathTrain)
idTrain=np.copy(pathTrain)
if IN_COLAB:
    for i in np.arange(len(idTrain)): idTrain[i]=idTrain[i][20:-4]
else:
    for i in np.arange(len(idTrain)): idTrain[i]=idTrain[i][13:-4]
#print(idTrain)
print('There are', len(idTrain), 'Train images')

In [ ]:
pathVal=glob.glob(path + "ValCropped/*.jpg")
idVal=np.copy(pathVal)
if IN_COLAB:
    for i in np.arange(len(idVal)): idVal[i]=idVal[i][18:-4]
else:
    for i in np.arange(len(idVal)): idVal[i]=idVal[i][11:-4]
#print(idVal)
print('There are', len(idVal) , 'Validation images')

In [ ]:
pathTest=glob.glob(path + "TestCropped/*.jpg")
idTest=np.copy(pathTest)
if IN_COLAB:
    for i in np.arange(len(idTest)): idTest[i]=idTest[i][19:-4]
else:
    for i in np.arange(len(idTest)): idTest[i]=idTest[i][12:-4]
#print(idTest)
print('There are', len(idTest) , 'Test images')

We can then plot an image with the related mask and contour.

In [ ]:
name_im = idTrain[10]
filename = path + 'TrainCropped/{}.jpg'.format(name_im)
image = imread(filename)
filename_Segmentation = path + 'TrainCropped/{}seg.png'.format(name_im)
image_Segmentation = imread(filename_Segmentation) # Value 0 or 255
image_Segmentation_boolean = (image_Segmentation/255).astype(np.uint8) # To get uint16
image_Segmentation_expand = np.expand_dims(image_Segmentation_boolean, axis=2)
image_mul_mask = (image_Segmentation_expand*image)
contours = cv2.findContours(image_Segmentation_boolean, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
contour = list(contours[0])
contour = np.squeeze(np.asarray(contour))

fig = plt.figure(figsize=(12, 12))
grid = AxesGrid(fig, 111,
                nrows_ncols = (1, 3),
                axes_pad = 0.5)
grid[0].imshow(image)
grid[0].scatter(contour[:,0],contour[:,1])
#grid[0].axis('off')
grid[0].set_title("Original image")
grid[1].imshow(image_Segmentation_boolean)
#grid[1].axis('off')
grid[1].set_title("Mask")
grid[2].imshow(image_mul_mask)
#grid[2].axis('off')
grid[2].set_title("Image with mask")

##Manual Feature Engineering

In this part, you will have to manually compute features relevant to the skin lesion classification. You can, for instance, implement the features described in [1] or in other papers.

**TODO**: Implement at least 10 features belonging to at least 3 classes of the ABCD rule, namely:
- Asymmetry
- Border
- Color (and Texture)
- Dimension (and Geometry)

**Please note the overall time (reading papers + implementation + computation) for computing the features. You will need it in the next part of the practical session.**

**This part counts for half of the grade of this TP**

[1] M. Zortea et al. "Performance of a dermoscopy-based computer vision system for the diagnosis of pigmented skin lesions compared with visual evaluation by experienced dermatologists". Artificial Intelligence in Medicine. 2014

In [ ]:
# XXXXXXXXXXXXX
# Add here your code

## Loading Metadata and Target values

You have at your disposal also two metadata, the age and the sex. If you want, you can use them as features in the classification but be careful ! There are missing values

We also load the target values (0 for benign and 1 for melanoma)

In [ ]:
Metatrain = pd.read_csv('./data/TrainCropped/ISIC-2017_Training_Data_metadata.csv')
print(Metatrain.head(10))
Groundtrain = pd.read_csv('./data/TrainCropped/ISIC-2017_Training_Part3_GroundTruth.csv')

In [ ]:
Ytrain=np.zeros(len(idTrain))
for i in range(len(idTrain)):
  name=idTrain[i]
  index=Groundtrain["image_id"].str.find(name)
  max_index = index.argmax()
  Ytrain[i]=int(Groundtrain["melanoma"][max_index])

In [ ]:
Metaval = pd.read_csv('./data/ValCropped/ISIC-2017_Validation_Data_metadata.csv')
Groundval = pd.read_csv('./data/ValCropped/ISIC-2017_Validation_Part3_GroundTruth.csv')

In [ ]:
Yval=np.zeros(len(idVal))
for i in range(len(idVal)):
  name=idVal[i]
  index=Groundval["image_id"].str.find(name)
  max_index = index.argmax()
  Yval[i]=int(Groundval["melanoma"][max_index])

In [ ]:
Metatest = pd.read_csv('./data/TestCropped/ISIC-2017_Test_v2_Data_metadata.csv')
Groundtest = pd.read_csv('./data/TestCropped/ISIC-2017_Test_v2_Part3_GroundTruth.csv')

In [ ]:
Ytest=np.zeros(len(idTest))
for i in range(len(idTest)):
  name=idTest[i]
  index=Groundtest["image_id"].str.find(name)
  max_index = index.argmax()
  Ytest[i]=int(Groundtest["melanoma"][max_index])

## Standard Machine Learning Prediction

In this part, you will use standard ML algorithms (such as logistic regression, SVM and Random Forests) on the features you previously computed.
Before starting, you should look at the data and check the proportion of classes.
Two hints:
- in sklearn you can use *class_weight='balanced'* when calling a ML method
- it exists a very interesting library called *imblearn* (e.g., from imblearn.over_sampling import ADASYN)

In [ ]:
np.random.seed(seed=666)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

**Pre-trained features**: for the next part, you can either use your own features or download a pre-computed set of features.

In [ ]:
labels=['f1_asym','f2_asym','f5_asym','f6_asym','f7_bord','f8_bord','f9_bord','f10_color','f11_color','f12_color','f13_color','f14_color','f15_color','f16_color','f17_color','f18_color','meanR','stdR','meanG','stdG','meanB','stdB','maxR','maxG','maxB','Area','Ecc','Diam','Ratio','Perim','f21_geom','f22_geom','f23_geom']

In [ ]:
!mkdir ./data
gdown.download(id="1ELg93G9jHeJOhCGBZ35HFs3NZr5Inj_j", output='./data/X_train_eng.npy', quiet=False)
train_computation_time=115 # in minute

In [ ]:
gdown.download(id="10sr56q1CEro7HPgQjYo0eVniGNA5og5H", output='./data/X_val_eng.npy', quiet=False)
test_computation_time=12 # in minute

In [ ]:
!mkdir ./data
gdown.download(id="1RbONb3WyG-CGt6SSi124TSXfkiCzb6-D", output='./data/X_test_eng.npy', quiet=False)
test_computation_time=38 # in minute

In [ ]:
Xtrain=np.load('./data/X_train_eng.npy')
Xtest=np.load('./data/X_test_eng.npy')
Xval=np.load('./data/X_val_eng.npy')

N,M=Xtrain.shape
print('Number of images: {0}; Number of features per image: {1}'.format(N,M))
print('Number of healthy nevus: {0}; Number of melanoma: {1}'.format(N-np.sum(Ytrain), np.sum(Ytrain)))

In [ ]:
# XXXXXXXXXXXXX
# Add here your code

**Question**: An important point in medical imaging is the explicability. Can you find the most important and discriminative features ? If yes, how ? Hint: random forest has a very interesting function...  

In [ ]:
# XXXXXXXXXXXXX
# Add here your code

Now you should have a final ML model with a generalization performance on the Test Set.

**Question**: Are you satisfied ? How long did it take to compute and evaluate the manually engineered features ? If you are not satisfied, what are the main problems ? Besides Deep Learning, could you do something to improve the results (if you had more time?)

XXXXXXXXXXXXX